## 寻找市场上的强势股

In [ ]:
import a1chemy.data_source as data_source
import pymongo

mongo_client = pymongo.MongoClient("mongodb://localhost:27017/", username='a1chemy', password='1B2C9046-E3CC-447F-9961-E125759BA44F')
mongo_ticks_client = data_source.MongoTicks(mongo_client)
mongo_tags_client = data_source.MongoTags(mongo_client)

def roc(ticks):
    def rate(t, day1, day2):
        return (t.close().iloc[day1] - t.close().iloc[day2])/t.close().iloc[day2]
    
    close = ticks.close().iloc[-1]
    short = ticks.close().iloc[-21]
    medium = ticks.close().iloc[-61]
    long = ticks.close().iloc[-121]
    return {
        'ticks': ticks,
        '1D': rate(ticks, -1, -2),
        '5D': rate(ticks, -1, -6),
        '1M': rate(ticks, -1, -20),
        'C/S': rate(ticks, -1, -21),
        'S/M': rate(ticks, -21, -61),
        'M/L': rate(ticks, -61, -121),
    }

In [ ]:
from tqdm.notebook import tqdm
stocks_tree = mongo_tags_client.tree(id='csi300')
stocks = []
for id, stock_tag in tqdm(stocks_tree.root.children.items()):
    exchange=id[0:2]
    symbol =id[3:]
    ticks = mongo_ticks_client.find_one(exchange=exchange, symbol=symbol)
    stocks.append(ticks)

In [ ]:
roc_list = []
for stock_tick in stocks:
    roc_list.append(roc(stock_tick))

In [ ]:
import operator
sorted_roc_list = sorted(roc_list, key = operator.itemgetter('C/S', 'S/M', 'M/L'))
for i in sorted_roc_list:
    print(('{}, {}, {:.4f}, {:.4f}, {:.4f}').format(i['ticks'].name, i['ticks'].symbol, i['C/S'], i['S/M'], i['M/L']))